In [1]:
import pandas as pd
from scipy.sparse import load_npz 

In [13]:
thresh = "0.7"
data_path = f"..\\data_output\\queries_thresh_{thresh}.parquet"
cw_repr_path = f"..\\data_output\\cw_thresh_{thresh}.npz"

In [14]:
data_df = pd.read_parquet(data_path)
data_df["expaded_words"] = data_df.apply(lambda row: set(row.cluwords_textual_repr).difference(set(row.text)), axis=1)

data_df.head(50)

,query_id,text,cluwords_textual_repr,expaded_words
0,156493,"[do, goldfish, grow]","[do, grow, develop, grows, goldfish]","{develop, grows}"
1,1110199,"[what, is, wifi, vs, bluetooth]","[wifi, bluetooth, becomes, seems, is, has, get...","{versus, vs., seems, are, has, gets, was, beco..."
2,1063750,"[why, did, the, us, volunterilay, enter, ww1]","[join, enter, entering, entered, did, could, d...","{ww2, how, could, entered, entering, does, of,..."
3,130510,"[definition, declaratory, judgment]","[declaratory, judgment, judgments, judgement, ...","{judgement, definitions, judgments}"
4,489204,"[right, pelvic, pain, causes]","[pelvic, anguish, discomfort, suffering, pain,...","{suffering, anguish, cause, discomfort, causing}"
5,573724,"[what, are, the, social, determinants, of, hea...","[health, a, in, the, of, political, economic, ...","{were, in, cultural, a, economic, societal, ha..."
6,168216,"[does, legionella, pneumophila, cause, pneumonia]","[doesn, did, does, legionella, pneumonia, infl...","{longbeachae, doesn, influenzae, sporogenes, c..."
7,1133167,"[how, is, the, weather, in, jamaica]","[weather, in, with, during, throughout, how, w...","{throughout, with, a, why, of, during, seems, ..."
8,527433,"[types, of, dysarthria, from, cerebral, palsy]","[from, athetoid, dysarthria, apraxia, dysproso...","{the, in, apraxia, type, dysprosody, athetoid,..."
9,1037798,"[who, is, robert, gray]","[blue, gray, brown, grey, becomes, seems, is, ...","{grey, blue, brown, seems, are, has, leonard, ..."


0                                                    {}
1                                         {versus, vs.}
2                                                    {}
3                   {judgement, definitions, judgments}
4                                                    {}
5                                                {were}
6                {catarrhalis, longbeachae, sporogenes}
7                                                    {}
8                                                    {}
9                                                {grey}
10                                              {could}
11                                                   {}
12    {decrease, minimize, could, increase, lessen, ...
13                                                   {}
14                                                   {}
15                                                   {}
16                                        {definitions}
17                                              

In [4]:
cw_csr_matrix = load_npz(cw_repr_path)

cw_csr_matrix.shape

(43, 9390)